In [1]:
import pandas as pd
import numpy as np

import plotly.express as px

import yaml

In [2]:
distances = pd.read_excel("../../data/ExtendedNuclearData.xlsx", sheet_name="Transport")

In [17]:
distances_dict = dict()
for idx, row in distances[distances["is_possible"] != 0].iterrows():
    org = row["from"]
    dest = row["to"]

    dist = row["costs"]

    if org in distances_dict.keys():
        relations_from_org = distances_dict[org]
        relations_from_org[dest] = dist
    else:
        relations_from_org = dict()
        relations_from_org[dest] = dist
        distances_dict[org] = relations_from_org

In [5]:
with open('C:/Users/ACER/Desktop/Uni/9_WiSe_23_24/OR-INF/operations_oppenheimer/operations_oppenheimer/src/parameter_config.yaml', 'r') as file:
    config_data = yaml.safe_load(file)

In [32]:
transport_costs_per_km = config_data["transport_costs"]["cost_factor"]
transport_factor = config_data["transport_costs"]["nuclear_factor"]

In [11]:
construction_costs = config_data["cisf"]["versions"]
versions = dict()

for cisf_dict in construction_costs:
    k = cisf_dict["size"]
    v = cisf_dict["costs"]
    versions[k] = v

In [16]:
snf = pd.read_csv("../../storage/n_hc_3/n_cisf_3/snf_shipped.csv")
nc = pd.read_csv("../../storage/n_hc_3/n_cisf_3/nc_shipped.csv")
cisf = pd.read_csv("../../storage/n_hc_3/n_cisf_3/cisf_build.csv")
hc = pd.read_csv("../../storage/n_hc_3/n_cisf_3/hc_build.csv")

##### Transport costs

In [42]:
snf_shipped = snf[snf["SNF"] != 0]
nc_shipped = nc[nc["NC"] != 0]

In [45]:
shipped = pd.concat([snf_shipped.rename(columns={"SNF": "amount"}), nc_shipped.rename(columns={"NC": "amount"})])

distances["key"] = distances["from"] + distances["to"]
shipped["key"] = shipped["from"] + shipped["to"]

shipped = pd.merge(left=shipped, left_on="key", right=distances.drop(columns=["from", "to", "is_possible"]), right_on="key", how="left").drop(columns="key")
shipped["transport_costs"] = shipped["costs"] * shipped["amount"] * transport_costs_per_km * transport_factor

total_transport_costs = np.sum(shipped["transport_costs"])

##### CISF costs

In [48]:
versions

{'small': 96444026.38,
 'medium': 142844431.51,
 'large': 189244836.64,
 'xlarge': 255531129.69,
 'complete': 341703310.66}

In [54]:
built = cisf[cisf["build"] == 1]

versions_built = built.groupby(by="size", as_index=False).agg(n = ("build", "sum"))

total_cisf_construction_costs = 0

for idx, row in versions_built.iterrows():
    total_cisf_construction_costs += row["n"] * versions[row["size"]]

##### Hot Cell costs

In [55]:
hc_construction_cost = config_data["hot_cells"]["construction_cost"]
processing_cost = config_data["hot_cells"]["processing_cost"]

In [71]:
total_hc_construction_cost = sum(hc["build"]) * hc_construction_cost[0]

In [72]:
total_processing_costs = np.sum(nc_shipped["NC"]) * processing_cost

##### Plotting

In [73]:
rows = []

rows.append({
    "cost_center": "transport",
    "amount": total_transport_costs
})

rows.append({
    "cost_center": "cisf",
    "amount": total_cisf_construction_costs
})

rows.append({
    "cost_center": "hc_construction",
    "amount": total_hc_construction_cost
})

rows.append({
    "cost_center": "hc_processing",
    "amount": total_processing_costs
})

costs = pd.DataFrame(rows)

In [76]:
px.bar(costs,
       x="cost_center",
       y="amount",
       template="seaborn",
       labels={"cost_center": "Cost center",
               "amount": "Amount [EUR]"})